In [12]:
import os
import random
from tqdm import tqdm
import numpy as np
import cv2

In [13]:
split_train_val_test = [0.85,0.05,0.1]

In [14]:

# path = "/media/fy/Data/DataSet/dataset/v2x_yolo/"
path = "/media/fy/Data/DataSet/dataset/v2x_v/"
image_paths = os.listdir(path+"images")
random.shuffle(image_paths)
total_num = len(image_paths)

split_train_val_test_num = [int(i*total_num) for i in split_train_val_test]

train_list = image_paths[:split_train_val_test_num[0]]
val_list = image_paths[split_train_val_test_num[0]:-split_train_val_test_num[2]]
test_list = image_paths[-split_train_val_test_num[2]:]


In [15]:
print("total_num",total_num)
print(len(train_list))
print(len(val_list))
print(len(test_list))


total_num 15285
12992
765
1528


In [16]:
with open(path + "./train.txt","w") as f:
    for p in train_list[:-1]:
        f.writelines("./images/"+p+"\n")
    f.writelines("./images/"+train_list[-1])

with open(path + "./val.txt","w") as f:
    for p in val_list[:-1]:
        f.writelines("./images/"+p+"\n")
    f.writelines("./images/"+val_list[-1])

with open(path + "./test.txt","w") as f:
    for p in test_list[:-1]:
        f.writelines("./images/"+p+"\n")
    f.writelines("./images/"+test_list[-1])

In [17]:
from ddd_utils import compute_box_3d, project_to_image, alpha2rot_y,rot_y2alpha
from ddd_utils import draw_box_3d, unproject_2d_to_3d

In [18]:
os.path.exists(path+"labels_old/")

False

In [21]:
label_dir = path+"labels/"
if not os.path.exists(path+"labels_old/"):
    os.system("mv {} {}; mkdir {}".format(label_dir,path+"labels_old/",label_dir))
label_dir = path+"labels_old/"
label_paths = os.listdir(label_dir)

bar = tqdm(label_paths)
DEBUG = False #True False


calib = np.array([[2186.359688 ,0.0, 968.712906, 0.0],
               [ 0.0, 2332.160319, 542.356703 ,0.0] ,
               [0.0 ,0.0, 1.0, 0]])

class_set = set()

for label_p in bar:
    labels = open(label_dir + label_p).readlines()
    image = (cv2.imread((label_dir + label_p).replace("txt","jpg").replace("labels_old","images"))) if DEBUG else None

    new_labels_path = (label_dir + label_p).replace("labels_old","labels")
    new_labels_file = open(new_labels_path,"w")
    for label in labels:
        tmp = label[:-1].split(' ')
        cat_id = int(tmp[0])
        truncated = int(float(tmp[1]))
        occluded = int(tmp[2])
        alpha = float(tmp[3])
        bbox = [float(tmp[4]), float(tmp[5]), float(tmp[6]), float(tmp[7])]
        dim = [float(tmp[8]), float(tmp[9]), float(tmp[10])]
        location = [float(tmp[11]), float(tmp[12]), float(tmp[13])]
        rotation_y = float(tmp[14])

        class_set.add(cat_id)

        # 计算 alpla
        x = (bbox[0] + bbox[2]) / 2
        alpha = rot_y2alpha(rotation_y, x, calib[0, 2], calib[0, 0])
        new_label = [cat_id]+bbox +location+ dim + [alpha]
        new_label_txt = ""
        for i in new_label:
            t = str(i) if type(i)==int else "{:.4f}".format(i)
            new_label_txt += t
            if not i is new_label[-1]:
                new_label_txt+= " "

        new_labels_file.writelines(new_label_txt)
        # print(new_label_txt)
        if not label is labels[-1]:
            new_labels_file.writelines("\n")


        if DEBUG:
            x = (bbox[0] + bbox[2]) / 2
            box_3d = compute_box_3d(dim, location, rotation_y)
            box_2d = project_to_image(box_3d, calib)
            image = draw_box_3d(image, box_2d)
            depth = np.array([location[2]], dtype=np.float32)
            pt_2d = np.array([(bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2],
                             dtype=np.float32)
            pt_3d = unproject_2d_to_3d(pt_2d, depth, calib)
            pt_3d[1] += dim[0] / 2

    if DEBUG:
        # from  matplotlib import pyplot as plt
        # plt.imshow(image)
        # plt.show()

        cv2.imshow('image', image)
        k = cv2.waitKey(1)
        if k==0:
            break


print(class_set)

100%|██████████| 15286/15286 [00:11<00:00, 1307.91it/s]

{0, 1, 2, 3, 4, 5, 6, 7, 9}


In [2]:
import numpy as np

In [3]:
def sigmoid(x):
    return 1.0/(1+np.exp(-x))

In [13]:
for i in range(100):
    t = i/100-0.5
    t *=10
    print(sigmoid(t)*200,t)

1.338570184856971 -5.0
1.4783082688563942 -4.9
1.6325142306319793 -4.8
1.8026597305695662 -4.699999999999999
1.9903603733808635 -4.6000000000000005
2.197388526118636 -4.5
2.4256869968548473 -4.4
2.677383565532956 -4.3
2.954806338654611 -4.2
3.260499874288186 -4.1000000000000005
3.597241992418312 -4.0
3.9680611468155007 -3.9000000000000004
4.376254187226095 -3.8
4.825404283533839 -3.7
5.319398715373173 -3.5999999999999996
5.862446150271264 -3.5
6.459092939690106 -3.3999999999999995
7.114237854527236 -3.3
7.8331445593528715 -3.2
8.621450988217223 -3.1
9.485174635513356 -3.0
10.430712615683543 -2.9000000000000004
11.464835179773745 -2.8000000000000003
12.594671211399296 -2.7
13.827684068669363 -2.6
15.17163600424871 -2.5
16.634539298784475 -2.4
18.22459220297123 -2.3
19.950097823937035 -2.1999999999999997
21.819364239122585 -2.1
23.84058440442351 -2.0
26.02169487259957 -1.9
28.370212980097563 -1.7999999999999998
30.89305301670695 -1.6999999999999997
33.596322973215116 -1.5999999999999996
